question 1.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the manipulator parameters
l1 = 1  # Link length 1
l2 = 1  # Link length 2
l3 = 1  # Link length 3

# Define the circle center coordinates
xc = 0  # X-coordinate of the circle center
yc = 0  # Y-coordinate of the circle center
r = 1.5  # Circle radius

# Define the forward kinematics function
def forward_kinematics(q):
    # Extract joint angles from the input array
    q1, q2, q3 = q

    # Calculate the end-effector coordinates
    x = xc + l1 * np.cos(q1) + l2 * np.cos(q1 + q2) + l3 * np.cos(q1 + q2 + q3)
    y = yc + l1 * np.sin(q1) + l2 * np.sin(q1 + q2) + l3 * np.sin(q1 + q2 + q3)

    return x, y

# Function to calculate the inverse kinematics for a given target point
def inverse_kinematics(target_x, target_y):
    # Calculate the first joint angle (theta1)
    q1 = np.arctan2(target_y, target_x)

    # Calculate the distance from the target point to the end-effector
    d = np.sqrt((target_x - xc)**2 + (target_y - yc)**2)

    # Calculate the second joint angle (theta2)
    q2 = np.arccos((l1**2 + l2**2 - d**2) / (2 * l1 * l2))

    # Calculate the third joint angle (theta3)
    q3 = np.arccos((d**2 + l3**2 - l2**2) / (2 * d * l3))

    return q1, q2, q3


# Generate points along a circle trajectory
num_points = 100
q = np.linspace(0, 2 * np.pi, num_points)
x_traj = r * np.cos(q) + xc
y_traj = r * np.sin(q) + yc

# Calculate the inverse kinematics for each point in the trajectory
joint_angles = []
for i in range(num_points):
    q1, q2, q3 = inverse_kinematics(x_traj[i], y_traj[i])
    joint_angles.append([q1, q2, q3])

# Convert joint angles from radians to degrees
joint_angles_deg = np.array(joint_angles) * 180 / np.pi

# Plot the trajectory of the end-effector
plt.plot(x_traj, y_traj, 'b-')

# Plot the circle
plt.plot(xc + r * np.cos(q), yc + r * np.sin(q), 'g-')

# Plot the manipulator for each joint angle configuration
for i in range(num_points):
    # Calculate the end-effector coordinates
    x_ee, y_ee = forward_kinematics(joint_angles[i])

    # Plot the manipulator links and end-effector
    plt.plot([0, l1 * np.cos(joint_angles[i][0])], [0, l1 * np.sin(joint_angles[i][0])], 'r-')
    plt.plot([l1 * np.cos(joint_angles[i][0]), l1 * np.cos(joint_angles[i][0]) + l2 * np.cos(joint_angles[i][1])], [l1 * np.sin(joint_angles[i][0]), l1 * np.sin(joint_angles[i][0]) + l2 * np.sin(joint_angles[i][1])], 'g-')
    plt.plot([l1 * np.cos(joint_angles[i][0]) + l2 * np.cos(joint_angles[i][1]), x_ee], [l1 * np.sin(joint_angles[i][0]) + l2 * np.sin(joint_angles[i][1]), y_ee], 'b-')



question 2.(a)

In [ ]:
import numpy as np
import math

class Robot:
    def __init__(self, link_lengths):
        self.link_lengths = link_lengths
        self.theta = np.zeros(3)

    def inverse_kinematics(self, x, y, z):
        """
        Computes the joint parameters for a given position in the Cartesian space.

        Args:
            x: The x-coordinate of the desired position.
            y: The y-coordinate of the desired position.
            z: The z-coordinate of the desired position.

        Returns:
            A numpy array containing the joint parameters.
        """

        theta1 = math.atan2(y, x)
        theta2 = math.acos((z**2 + self.link_lengths[0]**2 - self.link_lengths[1]**2) / (2 * self.link_lengths[0] * z))
        theta3 = math.atan2(math.sqrt(x**2 + y**2) * math.sin(theta2), z + self.link_lengths[0] * math.cos(theta2))

        return np.array([theta1, theta2, theta3])

    def forward_kinematics(self):
        """
        Computes the Cartesian position of the end-effector given the joint parameters.

        Returns:
            A numpy array containing the Cartesian position of the end-effector.
        """

        x = self.link_lengths[0] * math.cos(self.theta[0]) + self.link_lengths[1] * math.cos(self.theta[0] + self.theta[1]) + self.link_lengths[2] * math.cos(self.theta[0] + self.theta[1] + self.theta[2])
        y = self.link_lengths[0] * math.sin(self.theta[0]) + self.link_lengths[1] * math.sin(self.theta[0] + self.theta[1]) + self.link_lengths[2] * math.sin(self.theta[0] + self.theta[1] + self.theta[2])
        z = self.link_lengths[2] * math.sin(self.theta[2])

        return np.array([x, y, z])

# Define the robot parameters
link_lengths = np.array([0.25, 0.25, 0.25])

# Create a robot object
robot = Robot(link_lengths)

# Compute the joint parameters for each of the four corners of the manifold block
corner_joint_params = np.array([[robot.inverse_kinematics(0.45, 0.075, 0.1)],
                                [robot.inverse_kinematics(0.45, -0.075, 0.1)],
                                [robot.inverse_kinematics(0.25, -0.075, 0.1)],
                                [robot.inverse_kinematics(0.25, 0.075, 0.1)]])

# Substitute the joint parameters in the forward kinematics to verify the coordinates
corner_positions = np.array([robot.forward_kinematics(corner_joint_params[i]) for i in range(4)])

# Print the corner positions
print(corner_positions)


Question 2.(b)

In [ ]:
import numpy as np

def cubic_polynomial_interpolation(points, t_start, t_end):
    """
    Generates a cubic polynomial trajectory for the given points and time interval.

    Args:
        points: A list of points to interpolate between.
        t_start: The start time.
        t_end: The end time.

    Returns:
        A numpy array containing the Cartesian coordinates as functions of time.
    """

    num_points = len(points)
    num_coeffs = 4 * num_points

    # Construct the constraint matrix
    A = np.zeros((num_coeffs, num_coeffs))
    for i in range(num_points):
        t_i = (t_start + i * (t_end - t_start)) / (num_points - 1)
        A[4 * i, 0] = 1
        A[4 * i + 1, 1] = t_i
        A[4 * i + 2, 2] = t_i**2
        A[4 * i + 3, 3] = t_i**3

    # Construct the constraint vector
    b = np.zeros(num_coeffs)
    for i, point in enumerate(points):
        b[4 * i] = point[0]
        b[4 * i + 1] = point[1]
        b[4 * i + 2] = point[2]

    # Solve for the coefficients
    coeffs = np.linalg.lstsq(A, b, rcond=None)[0]

    # Generate the trajectory
    t = np.linspace(t_start, t_end, 100)
    cartesian_positions = []
    for t_val in t:
        x = coeffs[0] + coeffs[1] * t_val + coeffs[2] * t_val**2 + coeffs[3] * t_val**3
        y = coeffs[4] + coeffs[5] * t_val + coeffs[6] * t_val**2 + coeffs[7] * t_val**3
        z = coeffs[8] + coeffs[9] * t_val + coeffs[10] * t_val**2 + coeffs[11] * t_val**3
        cartesian_positions.append([x, y, z])

    return np.array(cartesian_positions)

# Define the points to interpolate between
points = np.array([[0.40, 0.06, 0.1],
                   [0.40, 0.01, 0.1],
                   [0.35, 0.01, 0.1],
                   [0.35, 0.06, 0.1]])

# Define the time interval
t_start = 0.0
t_end = 10.0

# Generate the trajectory
cartesian_positions = cubic_polynomial_interpolation(points, t_start, t_end)

# Print the trajectory
print(cartesian_positions)


Question 2.(c)

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

class Robot:
    def __init__(self, link_lengths, masses, inertias):
        self.link_lengths = link_lengths
        self.masses = masses
        self.inertias = inertias

    def compute_mass_matrix(self, q):
        M = np.zeros((3, 3))
        for i in range(3):
            M[i, i] = self.masses[i] + self.inertias[i]
            for j in range(i + 1, 3):
                M[i, j] = self.inertias[i * 3 + j] + self.inertias[j * 3 + i]
                M[j, i] = M[i, j]

        return M

    def compute_coriolis_and_centrifugal_matrix(self, q, qd):
        C = np.zeros((3, 3))
        for i in range(3):
            for j in range(i, 3):
                for k in range(3):
                    C[i, j] += self.inertias[i * 3 + k] * qd[k] * qd[j]
                    C[j, i] = C[i, j]

        return C

    def compute_gravity_torques(self, q):
        G = np.zeros(3)
        for i in range(3):
            for j in range(3):
                G[i] += self.masses[j] * 9.81 * math.sin(q[j])

        return G

    def inverse_dynamics(self, q, qd, qdd, cartesian_positions, cartesian_velocities, cartesian_accelerations):
        J = self.compute_jacobian(q)
        inv_J = np.linalg.pinv(J)

        desired_joint_accelerations = inv_J @ (
            cartesian_accelerations
            - np.cross(np.cross(J, qd), qd)
            - self.compute_coriolis_and_centrifugal_matrix(q, qd) @ qd
            - self.compute_gravity_torques(q)
        )

        feedforward_control_inputs = self.compute_mass_matrix(q) @ desired_joint_accelerations + self.compute_coriolis_and_centrifugal_matrix(q, qd) @ qd + self.compute_gravity_torques(q)

        return feedforward_control_inputs

    def compute_jacobian(self, q):
        J = np.zeros((3, 3))
        for i in range(3):
            for j in range(3):
                if i != j:
                    J[i, j] = 0
                else:
                    J[i, j] = -self.link_lengths[i] * math.
